<a href="https://colab.research.google.com/github/mateusduarte-max/Projetos_Python/blob/main/Reputacao_Nets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade openpyxl # atualização pacote excel
!pip install workadays # modulo para calcular data prometida e data limite de postagem
!pip install unidecode # modulo para retirar caracteres especiais

import pandas as pd
import re
import datetime
from workadays import workdays as wd
import numpy as np
import unidecode

# Bibliotecas Google Golab
from google.colab import auth
auth.authenticate_user()
# Importar biblioteca biqquery
from google.cloud import bigquery
# Bibliotecas Google Sheet
import gspread
from oauth2client.client import GoogleCredentials
from gspread_dataframe import set_with_dataframe
gc = gspread.authorize(GoogleCredentials.get_application_default())

# Montar Drive
from google.colab import drive
drive.mount('/content/drive')

# Modulo colab
!cp /content/drive/MyDrive/Colab_Notebooks/pacotes_modulos/modulos_colab.py /content

# Importar biblioteca biqquery
from modulos_colab import consulta_bigquery, grava_tabela_drive, busca_tabela_drive
#'maga-bigdata'
#'marketplace-analytics-333712'

In [ ]:
# Consulta reclamações formais

reclamacoes_formais = consulta_bigquery('maga-bigdata',"""SELECT W101.* ,
cast(a.PEDS_DAT_CAD as date) as Data_captado,cast(a.PEDS_DAT_FAT as date) as Data_faturado,cast(a.PEDS_DAT_PRZINI as date) as Data_Prazo_Entrega, tetrix.* except(code),a.PEDS_COD ,a.PEDS_EXT_COD,

CASE 
    WHEN UPPER(CAST(a.PEDS_COD as string)) like '%N%'  THEN 'Pedido Reverso'
    WHEN UPPER(CAST(a.PEDS_EXT_COD  as string)) like '%N%' then 'Pedido Reverso'
    WHEN UPPER(CAST(a.PEDS_COD as string)) like '%Z%'  THEN 'Pedido Reverso' 
    WHEN UPPER(CAST(a.PEDS_EXT_COD  as string)) like '%Z%' then 'Pedido Reverso'     
ELSE 'Não é Reverso' end as Pedido_Reverso, 

CASE
WHEN W101.LojaWebNome in ('ALLIANZ PARQUE','B2b Coorporativo','B2b Netshoes','CLUBE NETSHOES','Csu','CSU Passaporte Opte+','DIGI NETSHOES','Dotz','Freedom - Chapecoense','Freedom - Freelace','Freedom - Kappa','Freedom - Loja Cruzeiro','Freedom - Loja Internacional','Freedom - Loja Nba','Freedom - Netshoes','Freedom - Palmeiras','Freedom - Santos','Freedom - Sao Paulo','Freedom - Shoptimao','Freedom - Vasco','Func Netshoes','ITAU IUPP NETSHOES','Livelo','LOJA GAP','LOJA NFL','LOJA WSL','Ltm','MARKUP NETSHOES','Multiplus','Nba','NETSHOES','NIKE FUNCIONARIOS','NS2 - AMZ (Amazon)','NS2 - B2W','NS2 - CRF (Carrefour)','NS2 - MGL (Magazine Luiza)','NS2 - MLI (Mercado Livre)','NS2 - VVJ (ViaVarejo)','Shoestock','Shoestock Loja Fisica','SMILES NETSHOES','TUDO AZUL - NETSHOES','B2W','CRUZEIRO','Csu Banescard','Csu Banestes','Csu Banpará','Csu Banrisul','Csu Electrolux','Csu Pernambucanas','Csu Porto Seguro','Csu Vivo','Loja Do Itaú','MULTIPLUS NETSHOES MKTP','NS2 - BCP','PALMEIRAS','SANTOS','SÃO PAULO MANIA','SHOPTIMAO','Vasco') THEN 'Nets'
WHEN W101.LojaWebNome in ('CSU - SHOPPING ZATTINI','Csu Banrisul - Zattini','Csu Passaporte Opte+ - Zattini','Freedom - Zattini','Livelo Zattini','Ltm Zattini','MARKUP ZATTINI','Multiplus Zattini','SHOPFACIL ZATTINI','Smiles Zattini','TUDO AZUL - ZATTINI','ZATTINI','Csu Banescard - Zattini','Csu Banestes - Zattini','Csu Banpará - Zattini','Csu Electrolux - Zattini','Csu Pernambucanas - Zattini','Csu Porto Seguro - Zattini','Csu Vivo - Zattini','MULTIPLUS ZATTINI MKTP') THEN 'Zattini'
ELSE W101.LojaWebNome END as Depara_Grupo,

CASE 
WHEN W101.LojaWebNome in ('ALLIANZ PARQUE') THEN 'ALLIANZ PARQUE'
WHEN W101.LojaWebNome in ('Freedom - Chapecoense') THEN 'Chapecoense' 
WHEN W101.LojaWebNome in ('Freedom - Loja Cruzeiro','CRUZEIRO') THEN 'Cruzeiro' 
WHEN W101.LojaWebNome in ('Freedom - Freelace') THEN 'Freelace' 
WHEN W101.LojaWebNome in ('Func Netshoes') THEN 'Func Netshoes' 
WHEN W101.LojaWebNome in ('LOJA GAP') THEN 'GAP' 
WHEN W101.LojaWebNome in ('Freedom - Loja Internacional') THEN 'Internacional' 
WHEN W101.LojaWebNome in ('ITAU IUPP NETSHOES','Loja Do Itaú') THEN 'IUPP' 
WHEN W101.LojaWebNome in ('Freedom - Kappa') THEN 'Kappa' 
WHEN W101.LojaWebNome in ('Freedom - Loja Nba','Nba') THEN 'Nba' 
WHEN W101.LojaWebNome in ('LOJA NFL') THEN 'NFL' 
WHEN W101.LojaWebNome in ('NIKE FUNCIONARIOS') THEN 'NIKE FUNCIONARIOS' 
WHEN W101.LojaWebNome in ('Freedom - Palmeiras','PALMEIRAS') THEN 'Palmeiras' 
WHEN W101.LojaWebNome in ('Freedom - Santos','SANTOS') THEN 'Santos' 
WHEN W101.LojaWebNome in ('Freedom - Sao Paulo','SÃO PAULO MANIA') THEN 'Sao Paulo' 
WHEN W101.LojaWebNome in ('Shoestock Loja Fisica','Shoestock') THEN 'Shoestock' 
WHEN W101.LojaWebNome in ('Freedom - Shoptimao','SHOPTIMAO') THEN 'Shoptimao' 
WHEN W101.LojaWebNome in ('Freedom - Vasco','Vasco') THEN 'Vasco' 
WHEN W101.LojaWebNome in ('LOJA WSL') THEN 'WSL' 
WHEN W101.LojaWebNome in ('CSU - SHOPPING ZATTINI','Csu Banrisul - Zattini','Csu Passaporte Opte+ - Zattini','Freedom - Zattini','Livelo Zattini','Ltm Zattini','MARKUP ZATTINI','Multiplus Zattini','SHOPFACIL ZATTINI','Smiles Zattini','TUDO AZUL - ZATTINI','ZATTINI','Csu Banescard - Zattini','Csu Banestes - Zattini','Csu Banpará - Zattini','Csu Electrolux - Zattini','Csu Pernambucanas - Zattini','Csu Porto Seguro - Zattini','Csu Vivo - Zattini','MULTIPLUS ZATTINI MKTP') 
THEN 'Zattini' 
WHEN LojaWebNome in ('B2b Coorporativo','B2b Netshoes','CLUBE NETSHOES','Csu','CSU Passaporte Opte+','DIGI NETSHOES','Dotz','Freedom - Chapecoense','Freedom - Freelace','Freedom - Kappa','Freedom - Loja Cruzeiro','Freedom - Loja Internacional','Freedom - Loja Nba','Freedom - Netshoes','Freedom - Palmeiras','Freedom - Santos','Freedom - Sao Paulo','Freedom - Shoptimao','Freedom - Vasco','Func Netshoes','ITAU IUPP NETSHOES','Livelo','LOJA GAP','LOJA NFL','LOJA WSL','Ltm','MARKUP NETSHOES','Multiplus','Nba','NETSHOES','NIKE FUNCIONARIOS','NS2 - AMZ (Amazon)','NS2 - B2W','NS2 - CRF (Carrefour)','NS2 - MGL (Magazine Luiza)','NS2 - MLI (Mercado Livre)','NS2 - VVJ (ViaVarejo)','Shoestock','Shoestock Loja Fisica','SMILES NETSHOES','TUDO AZUL - NETSHOES','B2W','CRUZEIRO','Csu Banescard','Csu Banestes','Csu Banpará','Csu Banrisul','Csu Electrolux','Csu Pernambucanas','Csu Porto Seguro','Csu Vivo','Loja Do Itaú','MULTIPLUS NETSHOES MKTP','NS2 - BCP') THEN 'Nets' 
ELSE LojaWebNome END as Depara_Loja,

CASE 
WHEN lower(W101.Transportadora) in ('air cargo recife ltda') THEN 'Air Cargo LTDA'
WHEN lower(W101.Transportadora) in ('atlas transportes') THEN 'Atlas Transprote'
WHEN lower(W101.Transportadora) in ('carriers logistica e transportes ltda me') THEN 'Carriers Logística'
WHEN lower(W101.Transportadora) in ('empresa brasileira de correios e telegrafos') THEN 'Correios'
WHEN lower(W101.Transportadora) in ('dgt logistica e transportes ltda.','dgt logística e transportes ltda.') THEN 'DGT Logística'
WHEN lower(W101.Transportadora) in ('dialogo logistica inteligente ltda epp','diálogo logística inteligente ltda epp') THEN 'Díalogo Logística'
WHEN lower(W101.Transportadora) in ('direct express logistica integrada s/a','direct express logística integrada s/a') THEN 'Direct Express'
WHEN lower(W101.Transportadora) in ('dsl negocios e servicos especiais ltda','dsl negócios e serviços especiais ltda') THEN 'DSL Logística'
WHEN lower(W101.Transportadora) in ('ecolivery courieros me') THEN 'Ecolivery Logística'
WHEN lower(W101.Transportadora) in ('expresso jundiai logistica e transporte ltda') THEN 'Expresso Logística'
WHEN lower(W101.Transportadora) in ("faro's - transportes comercio importac?o e exportac?o ltda","faro's - transportes comércio importação e exportação ltda") THEN 'Faro Transportes'
WHEN lower(W101.Transportadora) in ('fast courier servicos ltda','fast courier serviços ltda') THEN 'Fast Courier LTDA'
WHEN lower(W101.Transportadora) in ('gfl logistica ltda') THEN 'GFL Logística'
WHEN lower(W101.Transportadora) in ('inside the time transportes ltda.') THEN 'Inside the Time LTDA'
WHEN lower(W101.Transportadora) in ('jadlog logistica ltda','jadlog logistica ltda.') THEN 'Jadlog'
WHEN lower(W101.Transportadora) in ('jamef transportes') THEN 'Jamef Transportes'
WHEN lower(W101.Transportadora) in ('l4b logistica ltda') THEN 'L4B Logística'
WHEN lower(W101.Transportadora) in ('lmtrans logistica em transportes ltda') THEN 'LMTrans Logística'
WHEN lower(W101.Transportadora) in ('logbee','expressa- logbee') THEN 'Logbee'
WHEN lower(W101.Transportadora) in ('magalog') THEN 'Magalog'
WHEN lower(W101.Transportadora) in ('nhl logistica inteligente ltda') THEN 'NHL Logística'
WHEN lower(W101.Transportadora) in ('on time express logistica e transportes ltda') THEN 'On time'
WHEN lower(W101.Transportadora) in ('rapid?o cometa logistica e transporte s/a','rapidão cometa logistica e transporte s/a','rapidão cometa logística e transporte s/a') THEN 'Rapidão Cometa'
WHEN lower(W101.Transportadora) in ('rbs - zero hora editora jornalistica s.a.') THEN 'RBS Transportes'
WHEN lower(W101.Transportadora) in ('sem transportadora') THEN 'Sem Transportadora'
WHEN lower(W101.Transportadora) in ('sequoia logistica e transportes sa','sequoia logística e transportes sa') THEN 'Sequoia Logística'
WHEN lower(W101.Transportadora) in ('total express') THEN 'Total Express'
WHEN lower(W101.Transportadora) in ('transfolha transporte e distribuic?o ltda','transfolha transporte e distribuicão ltda','transfolha transporte e distribuição ltda') THEN 'Transfolha'
WHEN lower(W101.Transportadora) in ('transportadora netshoes','netshoes - frota propria') THEN 'Transportadora Netshoes'
WHEN lower(W101.Transportadora) in ('air express fermac assessoria e consultoria em logistica e transporte ltda') THEN 'Air Express Logística'
WHEN lower(W101.Transportadora) in ('braspress transportes urgentes ltda') THEN 'Braspress Transportes'
WHEN lower(W101.Transportadora) in ('cajumar transportes ltda') THEN 'Cajumar Transportes'
WHEN lower(W101.Transportadora) in ('ctl logistica me') THEN 'CTL Logística'
WHEN lower(W101.Transportadora) in ('emerson rodrigo violin me') THEN 'Emerson Rodrigo Violin ME'
WHEN lower(W101.Transportadora) in ('gdr express network transportes ltda') THEN 'GDR Express'
WHEN lower(W101.Transportadora) in ('gps motos') THEN 'GPS Motos'
WHEN lower(W101.Transportadora) in ('irmãos silveira transportes em geral ltda') THEN 'Silveira Transportes'
WHEN lower(W101.Transportadora) in ('leonardo zenko pires de carvalho - me') THEN 'LEONARDO ZENKO PIRES DE CARVALHO - ME'
WHEN lower(W101.Transportadora) in ('lmtrans logística em transportes ltda') THEN 'LMTRANS Logística'
WHEN lower(W101.Transportadora) in ('movmov tecnologia da informacao ltda') THEN 'MovMov Tecnologias'
WHEN lower(W101.Transportadora) in ('nfn ferreira logistica em transportes de carga ltda') THEN 'NFN Logística'
WHEN lower(W101.Transportadora) in ('pacifico log transportes') THEN 'Pacífico Transportes'
WHEN lower(W101.Transportadora) in ('ramos transportes') THEN 'Ramos Transportes'
WHEN lower(W101.Transportadora) in ('ss reversa e projetos customizados ltda - epp') THEN 'SS Reversa'
WHEN lower(W101.Transportadora) in ('tex log') THEN 'Tex Log'
WHEN lower(W101.Transportadora) in ('transp. padrao mkp') THEN 'Transportadora Padrão MKP'
WHEN lower(W101.Transportadora) in ('transportadora route ltda') THEN 'Route Transportadora'
WHEN lower(W101.Transportadora) in ('tsv transportes rapidos') THEN 'TSV Transportes'
WHEN lower(W101.Transportadora) in ('transportadora shoestock') THEN 'Transportadora Shoestock'
WHEN lower(W101.Transportadora) in ('wf gomes transportes - me') THEN 'WF Transportes'
WHEN lower(W101.Transportadora) in ('empresa brasileira de transportes') THEN 'Empresa Brasileira de Transportes'
ELSE lower(W101.Transportadora) end as Depara_Transportadora,

FROM `maga-bigdata.sac.pc_abacos_sac` AS W101
LEFT JOIN `maga-bigdata.abacos_nets.aba_tcom_pedsai` as a on a.PEDS_COD = W101.Pedido
LEFT JOIN (SELECT DISTINCT o.code, ifnull(STRING_AGG(DISTINCT b.name,"/" order by b.name),'Pedido_1P') as seller_id_pedido, ifnulL(STRING_AGG(DISTINCT c.distribution_center_name,"/" order by c.distribution_center_name),'Sem CD Cadastrado') as cds_pedido,
           MIN(billing_date) as dt_pagamento_menor_pedido,MIN(dispatch_date) as dt_despacho_menor_pedido,MAX(o.sales_channel) as canal_venda
           FROM `maga-bigdata.tetrix.vw_order_sale` as o
           LEFT JOIN `maga-bigdata.tetrix.vw_order_shipping` as a on a.order_id = o.id
           LEFT JOIN `maga-bigdata.tetrix.vw_seller` as b on cast(b.code as string) = a.seller_id
           LEFT JOIN `maga-bigdata.tetrix.vw_shipping_product` as c on c.order_shipping_id = a.id
           GROUP BY 1)as tetrix on tetrix.code = a.PEDS_EXT_COD 
WHERE 1=1
AND AtendimentoOrigemNome in ('Jurídico Procon','Jurídico Cível')
AND W101.AtendimentoServicoNome  in ('Reclamações formais')
AND CAST(AtendimentoDataInicial as date) <= current_date()-30 """)

# Tratativa base
reclamacoes_formais.rename(columns={'seller_id_pedido':'LOJISTA'}, inplace=True)
reclamacoes_formais['LOJISTA'] = reclamacoes_formais['LOJISTA'].str.strip().str.lower().str.split('/',1).str[0].str.split('-', 1).str[0]
reclamacoes_formais_resumo = pd.pivot_table(reclamacoes_formais, index='LOJISTA', values='AtendimentoCodigo', aggfunc='count', margins=True)
reclamacoes_formais_resumo.rename(columns={'AtendimentoCodigo':'Qtd_Reclamacoes_Formais'}, inplace=True)

# Grava na tabela (Google Drive)
 
grava_tabela_drive('Base_reclamacoes_formais', reclamacoes_formais)
print("Bloco executado")


############################## Rodar quando a consulta não rodar #################################################
#reclamacoes_formais = busca_tabela_drive('Base_reclamacoes_formais')
#reclamacoes_formais.rename(columns={'seller_id_pedido':'LOJISTA'}, inplace=True)
#reclamacoes_formais['LOJISTA'] = reclamacoes_formais['LOJISTA'].str.strip().str.lower().str.split('/',1).str[0].str.split('-', 1).str[0]
#reclamacoes_formais_resumo = pd.pivot_table(reclamacoes_formais, index='LOJISTA', values='AtendimentoCodigo', aggfunc='count', margins=True)
#reclamacoes_formais_resumo.rename(columns={'AtendimentoCodigo':'Qtd_Reclamacoes_Formais'}, inplace=True)

In [ ]:
reclamacoes_formais = busca_tabela_drive('Base_reclamacoes_formais')
reclamacoes_formais.rename(columns={'seller_id_pedido':'LOJISTA'}, inplace=True)
reclamacoes_formais['LOJISTA'] = reclamacoes_formais['LOJISTA'].str.strip().str.lower().str.split('/',1).str[0].str.split('-', 1).str[0]
reclamacoes_formais_resumo = pd.pivot_table(reclamacoes_formais, index='LOJISTA', values='AtendimentoCodigo', aggfunc='count', margins=True)
reclamacoes_formais_resumo.rename(columns={'AtendimentoCodigo':'Qtd_Reclamacoes_Formais'}, inplace=True)

In [ ]:
# Dados pedidos com checkin
pedidos_ckeckin = consulta_bigquery('maga-bigdata',"""SELECT DISTINCT
                                                    A.PEDS_COD PEDIDO_INTERNO,
                                                    A.PEDS_EXT_COD PEDIDO_EXTERNO,
                                                    B.TROD_COD CODIGO_DA_TROCA,
                                                    B.TROD_STR_STA STATUS,
                                                    C.ITET_DAT_PROENT DATA_CHECKIN
                                                    FROM `maga-bigdata.abacos_nets.aba_tcom_pedsai` A
                                                    INNER JOIN `maga-bigdata.abacos_nets.aba_ttrd_trodev` B ON A.CLIF_COD = B.CLIF_COD
                                                    INNER JOIN `maga-bigdata.abacos_nets.aba_ttrd_itetrd` C ON B.TROD_COD = C.TROD_COD
                                                    --WHERE A.PEDS_COD IN ('XXX')
                                                    WHERE B.TROD_STR_STA = 'Finalizado'	
                                                        --AND PEDS_VAL_PED = ITET_VAL_PRETOT
                                                        --AND ITET_DAT_PROENT IS NOT NULL
                                                        AND B.TROD_DAT_PRAPRO >= '2021-09-01'""")

pedidos_ckeckin.rename(columns={'PEDIDO_INTERNO':'COD_PEDIDO', 'STATUS':'STATUS_checkin'}, inplace=True)

###################### Consulta pedidos nets bq #######################################################################################################
pedidos = consulta_bigquery('marketplace-analytics-333712',"""SELECT * FROM `marketplace-analytics-333712.data.pedidos_nets`
                                                                 where DATE(CAST(DAT_CRIA_PEDIDO AS DATE)) <= current_date()-30""")
###################### Retirada pedidos com checkin - Base original #########################################################################################
pedidos_checkin = pd.merge(pedidos,pedidos_ckeckin, how='left', on='COD_PEDIDO' )
base_pedidos = pedidos_checkin.query('STATUS_checkin!="Finalizado"')
###################### Tratativa base original ##############################################################################################################
# Retirar pedidos com T
base_pedidos['Validacao_pedidos'] = base_pedidos['COD_PEDIDO_LOJA'].str.contains("T")
# Formatar campos de data
base_pedidos['DATA_APROV'] = base_pedidos['DATA_APROV'].astype(object)
base_pedidos['DATA_APROV'] = pd.to_datetime(base_pedidos['DATA_APROV'], format='%Y%m%d')
base_pedidos['mes_data_aprovacao'] = base_pedidos['DATA_APROV'].dt.month
base_pedidos['DIAS_PRAZO'] = base_pedidos['DIAS_PRAZO'].replace('',0)
#base_pedidos['DAT_CRIA_PEDIDO'] = base_pedidos['DAT_CRIA_PEDIDO'].apply(lambda a: pd.to_datetime(a).date())
base_pedidos['DAT_CRIA_PEDIDO'] = pd.to_datetime(base_pedidos['DAT_CRIA_PEDIDO'], format='%Y%m%d')
base_pedidos['mes_data_pedido'] = base_pedidos['DAT_CRIA_PEDIDO'].dt.month
base_pedidos['DATA_PEDIDO'] = base_pedidos['DATA_PEDIDO'].astype(object)
base_pedidos['DATA_PEDIDO'] = pd.to_datetime(base_pedidos['DATA_PEDIDO'], format='%Y%m%d')
base_pedidos['DATA_ENTREGA'] = pd.to_datetime(base_pedidos['DATA_ENTREGA'])
base_pedidos['DATA_DESPACHO'] = pd.to_datetime(base_pedidos['DATA_DESPACHO'])
base_pedidos['Hoje'] = datetime.datetime.today().strftime('%Y-%m-%d')
base_pedidos['Hoje'] = pd.to_datetime(base_pedidos['Hoje'], format='%Y-%m-%d')
#Formatar colunas Lojista e Id seller
base_pedidos['STATUS'] = base_pedidos['STATUS'].str.strip()
base_pedidos['LOJISTA'] = base_pedidos['LOJISTA'].str.lower()
base_pedidos['ID_SELLER'] = base_pedidos['ID_SELLER'].str.strip()
base_pedidos['ID_SELLER'] = base_pedidos['ID_SELLER'].astype(str)

# Data Prometida
def data_prometida(base_pedidos):
  return wd.workdays(base_pedidos['DATA_APROV'], base_pedidos['DIAS_PRAZO'], country=None)
base_pedidos['DATA_PROMETIDA'] = base_pedidos.apply(data_prometida, axis=1)
base_pedidos['mes_data_prometida'] = base_pedidos['DATA_PROMETIDA'].dt.month

# Sellers blocado
sellers_blocado = ['13514',
                  '7968',
                  '9321',
                  '3328',
                  '1483',
                  '4761',
                  '6749',
                  '13590',
                  '12806',
                  '4901',
                  '5382',
                  '2281',
                  '12114',
                  '3468',
                  '10641',
                  '4282',
                  '3441',
                  '5021',
                  '5941',
                  '581',
                  '9963',
                  '13268',
                  '4188',
                  '7930',
                  '12666',
                  '12739',
                  '4764',
                  '7861',
                  '12865',
                  '11932',
                  '7644',
                  '12089',
                  '3464',
                  '6721',
                  '13201',
                  '2082',
                  '3421',
                  '12814',
                  '12799',
                  '5601',
                  '7934',
                  '12855',
                  '11956',
                  '12130',
                  '9501',
                  '4230',
                  '12844',
                  '14699',
                  '581',
                  '14673',
                  '14630',
                  '13268',
                  '14278',
                  '14587',
                  '14887',
                  '14550',
                  '14970',
                  '15182',
                  '14897',
                  '15057',
                  '14967',
                  '1356',
                  '3131',
                  '15237',
                  '15173',
                  '13002',
                  '15244',
                  '15116',
                  '15574',
                  '15510',
                  '15269',
                  '14697',
                  '3385',
                  '15812',
                  '16164',
                  '15971',
                  '16243',
                  '15206',
                  '16273',
                  '16116',
                  '14468',
                  '16755',
                  '17115',
                  '14755',
                  '17008',
                  '16151',
                  '17274',
                  '17471']
#Função para coluna blocado
def seller_blocado(base_pedidos):
  if (base_pedidos['ID_SELLER'] in sellers_blocado):
    return 'sim'
  else:
    return 'não' 
# Coluna Blocado
base_pedidos['Blocado'] = base_pedidos.apply(seller_blocado, axis=1)

################################################################# Base pedidos  ###################################################################
# Base Pedidos 
hoje = pd.datetime.now().date()
base_pedidos_mes = base_pedidos.query('Validacao_pedidos==False & DATA_PROMETIDA<="{}"'.format(hoje)).drop_duplicates()
# Total pedidos
base_seller_lojista = base_pedidos_mes[['ID_SELLER','LOJISTA']]
base_tt_pedidos = pd.DataFrame(pd.pivot_table(base_pedidos_mes, index=['ID_SELLER','LOJISTA'], columns='STATUS', values='COD_PEDIDO', aggfunc='count',margins=True, margins_name='TT_Pedidos')).fillna(0)

################################################################## Pedidos Entregues ################################################################
# Pedidos Entregues
base_pedidos_entregues = base_pedidos_mes.query('STATUS=="ENTREGUE"')
def analise_pedidos(base_pedidos_entregues):
  if base_pedidos_entregues['DATA_ENTREGA'] <= base_pedidos_entregues['DATA_PROMETIDA']:
    return 'entregue_dentro_prazo'
  else:
    return 'entregue_fora_prazo'
base_pedidos_entregues['status_entrega'] = base_pedidos_entregues.apply(analise_pedidos, axis=1)
resumo_base_pedidos_entregues = pd.DataFrame(pd.pivot_table(base_pedidos_entregues, index=['ID_SELLER','LOJISTA'], columns='status_entrega', values='COD_PEDIDO', aggfunc='count').fillna(0))
##################################################################3 Concatenação 1 ####################################################################

concatenacao_1 = pd.merge(base_tt_pedidos,resumo_base_pedidos_entregues, how='left' , on=['ID_SELLER','LOJISTA']).drop_duplicates()

##################################################################### Pedidos Despachados ################################################################

# Base pedidos despachado
base_pedidos_despachado = base_pedidos_mes.query('STATUS==["DESPACHADO","ENTREGUE"] & DATA_DESPACHO!=""').drop_duplicates()
# Função limite de postagem
def data_limite_postagem(base_pedidos_despachado):
  if base_pedidos_despachado['Blocado'] == 'não':
    return wd.workdays(base_pedidos_despachado['DATA_APROV'], 2 , country=None)
  elif base_pedidos_despachado['Blocado'] == 'sim': 
    return wd.workdays(base_pedidos_despachado['DATA_APROV'], 5 , country=None)
base_pedidos_despachado['DATA_LIMITE_POSTAGEM'] = base_pedidos_despachado.apply(data_limite_postagem, axis=1)

# Função análise postagem
def analise_postagem(base_pedidos_despachado):
  if base_pedidos_despachado['DATA_DESPACHO'] <= base_pedidos_despachado['DATA_LIMITE_POSTAGEM']:
    return 'postagem_dentro_prazo'
  else:
    return 'postagem_fora_prazo'
base_pedidos_despachado['status_postagem'] = base_pedidos_despachado.apply(analise_postagem, axis=1)
resumo_base_pedidos_despachado = pd.DataFrame(pd.pivot_table(base_pedidos_despachado, index=['ID_SELLER','LOJISTA'], columns='status_postagem', values='COD_PEDIDO', aggfunc='count').fillna(0))

##################################################################3 Concatenação 2 ####################################################################

concatenacao_2 = pd.merge(concatenacao_1,resumo_base_pedidos_despachado, how='left' , on=['ID_SELLER','LOJISTA']).drop_duplicates()

##################################################################### Pedidos Pendentes ################################################################
# Base Pedidos Faturados
base_pedidos_faturado = base_pedidos_mes.query('STATUS=="FATURADO"')
# Função limite postagem
def data_limite_postagem(base_pedidos_faturado):
  if base_pedidos_faturado['Blocado'] == 'não':
    return wd.workdays(base_pedidos_faturado['DATA_APROV'], 2 , country=None)
  elif base_pedidos_faturado['Blocado'] == 'sim': 
    return wd.workdays(base_pedidos_faturado['DATA_APROV'], 5 , country=None)
base_pedidos_faturado['DATA_LIMITE_POSTAGEM'] = base_pedidos_faturado.apply(data_limite_postagem, axis=1)

# Base Pedidos Pronto para faturar
base_pedidos_pronto_faturar = base_pedidos_mes.query('STATUS=="PRONTO PARA FATURAR"')
# Função limite postagem
def data_limite_postagem(base_pedidos_pronto_faturar):
  if base_pedidos_pronto_faturar['Blocado'] == 'não':
    return wd.workdays(base_pedidos_pronto_faturar['DATA_APROV'], 2 , country=None)
  elif base_pedidos_pronto_faturar['Blocado'] == 'sim': 
    return wd.workdays(base_pedidos_pronto_faturar['DATA_APROV'], 5 , country=None)
base_pedidos_pronto_faturar['DATA_LIMITE_POSTAGEM'] = base_pedidos_pronto_faturar.apply(data_limite_postagem, axis=1)

# Concatenação bases
pedidos_pendentes = pd.concat([base_pedidos_faturado, base_pedidos_pronto_faturar]).drop_duplicates()


# Coluna ontem
pedidos_pendentes['Ontem'] = pedidos_pendentes['Hoje'] - datetime.timedelta(days = 1)
# Coluna Pedidos Atrasados
def pedidos_em_atraso(pedidos_pendentes):
  if pedidos_pendentes['DATA_PROMETIDA'] <= pedidos_pendentes['Ontem']: 
    return 'Pedido_Atrasado'
  else:
    return ''
pedidos_pendentes['Pedidos_Atrasados'] =  pedidos_pendentes.apply(pedidos_em_atraso, axis=1)   

# Coluna Expedição  Atrasada
def expedicao_em_atraso(pedidos_pendentes):
  if pedidos_pendentes['DATA_LIMITE_POSTAGEM'] <= pedidos_pendentes['Ontem']: 
    return 'Expedicao_Atrasada'
  else:
    return ''
pedidos_pendentes['Pedidos_Expedicao_Atrasada'] =  pedidos_pendentes.apply(expedicao_em_atraso, axis=1)

base1 = pedidos_pendentes.query('Pedidos_Atrasados=="Pedido_Atrasado"').drop_duplicates(subset=['ID_SELLER', 'COD_PEDIDO_LOJA'])
resumo_base1 = pd.pivot_table(base1, index='ID_SELLER', values=['Pedidos_Atrasados'], aggfunc='count')
base2 = pedidos_pendentes.query('Pedidos_Expedicao_Atrasada=="Expedicao_Atrasada"').drop_duplicates(subset=['ID_SELLER', 'COD_PEDIDO_LOJA'])
resumo_base2 = pd.pivot_table(base2, index='ID_SELLER', values=['Pedidos_Expedicao_Atrasada'], aggfunc='count')

######################################################################### Dados KA Nets ######################################################################################
dados_ka = consulta_bigquery('maga-bigdata', """select * from maga-bigdata.nets_gestao_info.tgi_seller_group""")
dados_ka.rename(columns={'group_owner':'Assistido', 'code':'ID_SELLER'}, inplace=True)
dados_ka['Assistido'] = dados_ka['Assistido'].str.replace('Não Assistido', 'Não_Assistido')
# Função sellers ka
def sellers_ka(dados_ka):
  if dados_ka['Assistido'] == 'Assistido':
    return "Key Account"
  else:
    return 'Long Tail'
dados_ka['Key Account?']  = dados_ka.apply(sellers_ka, axis=True)
dados_ka_final = dados_ka[['ID_SELLER', 'Key Account?']]
dados_ka_final['ID_SELLER'] = dados_ka_final['ID_SELLER'].apply(str)
dados_ka_final['ID_SELLER'] = dados_ka_final['ID_SELLER'].str.strip()

##################################################################3 Concatenação 3 e 4 ####################################################################################

concatenacao_3 = pd.merge(concatenacao_2, base_seller_lojista, how='left', on=['ID_SELLER','LOJISTA']).drop_duplicates()
concatenacao_4 = pd.merge(concatenacao_3, dados_ka_final, how='left', on=['ID_SELLER']).drop_duplicates()
######################################################################### Dados Sellers NSE #################################################################################

base_nse = consulta_bigquery('marketplace-analytics-333712', """SELECT * FROM `marketplace-analytics-333712.data.sellers_nets_nse`""")
base_nse.rename(columns={'Seller_ID':'ID_SELLER'}, inplace=True)
# Função sellers nse
def seller_nse(base_nse):
  if ('NETSHOES_ENTREGAS' in base_nse):
    return 'NSE'
  else:
    return 'Não'
base_nse['Ns_Entregas?'] = base_nse['Tipo_Gateway_Frete'].apply(seller_nse)
base_nse_final = base_nse[['ID_SELLER', 'Ns_Entregas?']]
base_nse_final['ID_SELLER'] = base_nse_final['ID_SELLER'].apply(str)
##################################################################3 Concatenação 5 ####################################################################################

concatenacao_5 = pd.merge(concatenacao_4, base_nse_final, how='left', on=['ID_SELLER']).drop_duplicates()

######################################################################### Dados CR e Nps #################################################################################

dados_cr_nps = consulta_bigquery('maga-bigdata', """SELECT Seller_id,
                                                    round((SAFE_DIVIDE((SUM(abac_App_Site)+SUM(abac_B2B)+SUM(abac_Cadastro)+SUM(abac_Campanhas)+SUM(abac_Entrega)+SUM(abac_Ncard)+SUM(abac_Pedido)+SUM(abac_Produto)+SUM(abac_TVC)+SUM(abac_Outros)),SUM(Captado))),3) as CR_Seller,
                                                    round((SAFE_DIVIDE((SUM(nps_promotor) - SUM(nps_detrator)),(sum(nps_detrator)+SUM(nps_promotor)+SUM(nps_neutro)))*100),2) as NPS_Seller
                                                    FROM maga-bigdata.sac_inteligencia_clientes.sellers_netshoes_indicadores
                                                        WHERE seller_id is not null
                                                        AND data <= current_date()-30
                                                        GROUP BY Seller_id""")

#dados_cr_nps['CR_Seller'] = round((dados_cr_nps['CR_Seller']*100),2)
dados_cr_nps['CR_Seller'] = pd.to_numeric(dados_cr_nps['CR_Seller']).fillna(0)
dados_cr_nps['NPS_Seller'] = pd.to_numeric(dados_cr_nps['NPS_Seller']).fillna(0)
dados_cr_nps['LOJISTA'] = dados_cr_nps['Seller_id'].str.lower()
dinamica_cr_nps = dados_cr_nps[['LOJISTA', 'CR_Seller', 'NPS_Seller']].drop_duplicates(subset='LOJISTA')

##################################################################3 Concatenação 6 ####################################################################################

concatenacao_6 = pd.merge(concatenacao_5, dinamica_cr_nps , how='left', on=['LOJISTA']).drop_duplicates()
######################################################################### Base Hubsales #################################################################################
# Import base
#'/content/drive/MyDrive/Ciclo Pedidos Nets Novo/sellers Hubsales - BF.xlsx'
base_husales = pd.read_excel('/content/drive/MyDrive/dados_pedidos/Nets/Ciclo_pedido/sellers Hubsales - BF.xlsx', usecols=['Id Seller', 
                                                                                                                           'Nome Loja', 
                                                                                                                           'Hubsales?']).fillna(0)
# Ajustes colunas
base_husales.rename(columns={'Id Seller':'ID_SELLER', 'Nome Loja':'LOJISTA'}, inplace=True)
base_husales['ID_SELLER'] = base_husales['ID_SELLER'].apply(int).apply(str)
base_husales['LOJISTA'] = base_husales['LOJISTA'].str.lower()
base_husales_final = base_husales[['ID_SELLER', 'Hubsales?']]

##################################################################3 Concatenação 7 ####################################################################################
concatenacao_7 = pd.merge(concatenacao_6, base_husales_final , how='left', on=['ID_SELLER']).drop_duplicates()
concatenacao_7['Hubsales?'] = concatenacao_7['Hubsales?'].fillna('Não')

######################################################################### Concatenação 8 e 9 #################################################################################

concatencao_8 = pd.merge(concatenacao_7,resumo_base1, how='left', on='ID_SELLER').drop_duplicates()
concatencao_8['Pedidos_Atrasados'] = concatencao_8['Pedidos_Atrasados'].fillna(0)
concatencao_9 = pd.merge(concatencao_8,resumo_base2, how='left', on='ID_SELLER').drop_duplicates()
concatencao_9['Pedidos_Expedicao_Atrasada'] = concatencao_9['Pedidos_Expedicao_Atrasada'].fillna(0)

######################################################################### Base GMV ###################################################################################################
base_pedidos_mes_gmv = pd.pivot_table(base_pedidos_mes, index='ID_SELLER', values='VLR', aggfunc='sum')
base_pedidos_mes_gmv.rename(columns={'VLR':'GMV'}, inplace=True)

######################################################################### Concatenação 10 ###########################################################################################

concatencao_10 = pd.merge(concatencao_9, base_pedidos_mes_gmv, how='left', on='ID_SELLER').drop_duplicates()
concatencao_10['GMV'] = round(concatencao_10['GMV'],2)

######################################################################### Concatenação 11 ###########################################################################################

concatenacao_11 = pd.merge(concatencao_10, reclamacoes_formais_resumo , how='left', on=['LOJISTA']).drop_duplicates()
concatenacao_11['Qtd_Reclamacoes_Formais'] = concatenacao_11['Qtd_Reclamacoes_Formais'].fillna(0)

######################################################################### Indicador de reputação #####################################################################################
concatenacao_11['% Entrega'] = round((concatenacao_11['entregue_dentro_prazo']/concatenacao_11['TT_Pedidos'])*100,2).fillna(0)
concatenacao_11['% Postagem'] = round((concatenacao_11['postagem_dentro_prazo']/concatenacao_11['TT_Pedidos'])*100,2).fillna(0)
concatenacao_11['% Cancelados'] = round((concatenacao_11['CANCELADO']/concatenacao_11['TT_Pedidos'])*100,2).fillna(0)
concatenacao_11['% Reclamação'] = round((concatenacao_11['Qtd_Reclamacoes_Formais']/concatenacao_11['TT_Pedidos'])*100,2).fillna(0)

# Classificação  regua entrega, despacho, cancelados, reclamação e nps
def regra_entrega(concatenacao_11):
  if concatenacao_11['% Entrega'] >= 0.0 and concatenacao_11['% Entrega'] <= 84.99:
    return 1
  elif concatenacao_11['% Entrega'] >= 85 and concatenacao_11['% Entrega'] <= 88.99:
    return 2
  elif concatenacao_11['% Entrega'] >= 89 and concatenacao_11['% Entrega'] <= 92.99:
    return 3
  elif concatenacao_11['% Entrega'] >= 93 and concatenacao_11['% Entrega'] <= 94.99:
    return 4
  elif concatenacao_11['% Entrega'] >= 95:
    return 5
concatenacao_11['reg_entrega'] = concatenacao_11.apply(regra_entrega, axis=1)


def regra_despacho(concatenacao_11):
  if concatenacao_11['% Postagem'] >= 0.0 and concatenacao_11['% Postagem'] <= 84.99:
    return 1
  elif concatenacao_11['% Postagem'] >= 85 and concatenacao_11['% Postagem'] <= 88.99:
    return 2
  elif concatenacao_11['% Postagem'] >= 89 and concatenacao_11['% Postagem'] <= 92.99:
    return 3
  elif concatenacao_11['% Postagem'] >= 93 and concatenacao_11['% Postagem'] <= 94.99:
    return 4
  elif concatenacao_11['% Postagem'] >= 95:
    return 5
concatenacao_11['reg_Postagem'] = concatenacao_11.apply(regra_despacho, axis=1)


def regra_cancelados(concatenacao_11):
  if concatenacao_11['% Cancelados'] <= 2.0:
    return 5
  elif concatenacao_11['% Cancelados'] >= 2.01 and concatenacao_11['% Cancelados'] <= 3.0:
    return 4
  elif concatenacao_11['% Cancelados'] >= 3.01 and concatenacao_11['% Cancelados'] <= 4.0:
    return 3
  elif concatenacao_11['% Cancelados'] >= 4.01 and concatenacao_11['% Cancelados'] <= 5.0:
    return 2
  elif concatenacao_11['% Cancelados'] >= 5.01:
    return 1
concatenacao_11['reg_Cancelados'] = concatenacao_11.apply(regra_cancelados, axis=1)


def regra_reclamacao(concatenacao_11):
  if concatenacao_11['% Reclamação'] <= 1.0:
    return 5
  elif concatenacao_11['% Reclamação'] >= 1.01 and concatenacao_11['% Reclamação'] <= 2.50:
    return 4
  elif concatenacao_11['% Reclamação'] >= 2.51 and concatenacao_11['% Reclamação'] <= 3.50:
    return 3
  elif concatenacao_11['% Reclamação'] >= 3.51 and concatenacao_11['% Reclamação'] <= 4.00:
    return 2
  elif concatenacao_11['% Reclamação'] >= 4.01:
    return 1
concatenacao_11['reg_Reclamação'] = concatenacao_11.apply(regra_cancelados, axis=1)


def regra_nps(concatenacao_11):
  if concatenacao_11['NPS_Seller'] <= 64.9 :
    return 1
  elif concatenacao_11['NPS_Seller'] >= 65.00 and concatenacao_11['NPS_Seller'] <= 67.99:
    return 2
  elif concatenacao_11['NPS_Seller'] >= 68.00 and concatenacao_11['NPS_Seller'] <= 70.00:
    return 3
  elif concatenacao_11['NPS_Seller'] >= 70.01 and concatenacao_11['NPS_Seller'] <= 74.99:
    return 4
  elif concatenacao_11['NPS_Seller'] >= 75.00:
    return 5
concatenacao_11['reg_nps'] = concatenacao_11.apply(regra_nps, axis=1)


def regra_cr(concatenacao_11):
  if concatenacao_11['CR_Seller'] >= 0.301 :
    return 1
  elif concatenacao_11['CR_Seller'] >= 0.201 and concatenacao_11['CR_Seller'] <= 0.300:
    return 2
  elif concatenacao_11['CR_Seller'] >= 0.151 and concatenacao_11['CR_Seller'] <= 0.200:
    return 3
  elif concatenacao_11['CR_Seller'] >= 0.101 and concatenacao_11['CR_Seller'] <= 0.150:
    return 4
  elif concatenacao_11['CR_Seller'] <= 0.100:
    return 5
concatenacao_11['reg_cr'] = concatenacao_11.apply(regra_cr, axis=1)


# Indicador da reputação
def funcao_reputacao(concatenacao_11):
  if concatenacao_11['Ns_Entregas?'] == 'Não':
    return round((concatenacao_11['reg_nps']/5*1.45)+(concatenacao_11['reg_entrega']/5*1.50)+(concatenacao_11['reg_cr']/5*0.50)+(concatenacao_11['reg_Reclamação']/5*0.35)+(concatenacao_11['reg_Cancelados']/5*1.20),2)
  elif concatenacao_11['Ns_Entregas?'] == 'NSE':
    return round((concatenacao_11['reg_nps']/5*1.45)+(concatenacao_11['reg_Postagem']/5*1.50)+(concatenacao_11['reg_cr']/5*0.50)+(concatenacao_11['reg_Reclamação']/5*0.35)+(concatenacao_11['reg_Cancelados']/5*1.20),2)
concatenacao_11['Indicador_Reputacao'] = concatenacao_11.apply(funcao_reputacao, axis=1)


concatenacao_11.head(1)



In [ ]:
# Tabela Dash - BigQuery
# renomear colunas para BigQuery

concatenacao_11.rename(columns={'PRONTO PARA FATURAR':'Pronto_para_faturar',
                                'Key Account?':'Key_Account',
                                'Hubsales?':'Hubsales',
                                'Ns_Entregas?':'Ns_Entregas',
                                '% Entrega':'P_Entrega',
                                '% Postagem':'P_Postagem',
                                '% Cancelados':'P_Cancelados',
                                '% Reclamação':'P_Reclamacao',
                                'reg_Reclamação':'reg_Reclamacao'}, inplace=True)

concatenacao_12 = concatenacao_11.query('ID_SELLER!="TT_Pedidos"')


# Gravar tabela
concatenacao_12.to_gbq(destination_table='marketplace_analytics.dados_reputacao_nets',project_id='maga-bigdata', if_exists='replace')
print('Tabela atualizada!')
# Gravar tabela
pedidos_pendentes.to_gbq(destination_table='marketplace_analytics.dados_pedidos_pendentes_nets',project_id='maga-bigdata', if_exists='replace')
print('Tabela atualizada!')
